# AgroPulse Chatbot

In [227]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.3-70b-versatile")

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F480D49F30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F480D49CF0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [211]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads .env variables into environment

hf_token = os.getenv("HF_TOKEN")
os.environ["HF_TOKEN"] = hf_token


In [212]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [213]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [214]:
from bs4 import BeautifulSoup, SoupStrainer

In [215]:
import bs4


def load_web_docs(urls):
    all_docs = []
    for url in urls:
        loader = WebBaseLoader(
            web_paths=(url,),
            bs_kwargs=dict(
                parse_only=bs4.SoupStrainer(
                    class_=("post-content", "post-title", "post-header")
                )
            ),
        )
        docs = loader.load()
        all_docs.extend(docs)
    return all_docs

In [216]:
urls = [
    "https://invadeagro.com/2025/07/03/common-crop-diseases-india/",
    "https://eos.com/blog/crop-diseases/",
    "https://www.growag.com/highlights/article/guide-to-crop-disease-and-management",
    "https://kshema.co/blogs/common-crop-diseases-and-how-to-prevent-them/",
    "https://krikso.co.in/pest-and-disease-management-protecting-your-crops-effectively/",
    "https://blog.plantwise.org/tag/plant-diseases/",
    "https://farmonaut.com/blogs/best-farming-organic-farming-blogs-in-india",
    "https://agrisearchindia.com/en/blog"
]

docs = load_web_docs(urls)
print(f"Total docs: {len(docs)}")

Total docs: 8


In [217]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
vectorstore=Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever(search_kwargs={"k": 3})
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001F482791AB0>, search_kwargs={'k': 3})

In [218]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

## Prompt Template
system_prompt = (
    "You are an expert assistant for answering crop and farming questions. "
    "Use the following retrieved context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Always respond in the same language as the user's question. "
    "Provide a detailed, thorough, and informative answer, covering all relevant points. "
    "Explain technical terms if needed.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Please answer in detail {input}"),
    ]
)

In [219]:
## Arxiv--Research
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [220]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [221]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool
math_chain=LLMMathChain.from_llm(llm=llm)
calculator=Tool(
    name="Calculator",
    func=math_chain.run,
    description="A calculator for farming math. Use this when farmers ask about quantities, costs, yields, fertilizer requirements, or any arithmetic calculation."
)

In [222]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

In [223]:
# Wrap retriever as a tool
def run_qa_chain_tool(question):
    try:
        response = chain.invoke({"input": question})
        if isinstance(response, dict) and "result" in response:
            return response["result"]
        return response
    except Exception as e:
        return f"An error occurred: {e}"



retriever_tool = Tool(
    name="CropDocs",
    func=run_qa_chain_tool,
    description="Useful for answering farming and crop disease questions from AgroPulse blog database"
)

# Tools list (retriever + wiki + arxiv)
tools = [retriever_tool, wiki, calculator]

from langchain.agents import initialize_agent, Tool, AgentType
agent = initialize_agent(
    tools,
    llm,   
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [226]:
print(agent.run("What are the symptoms of rice blast disease?"))



> Entering new AgentExecutor chain...
To find the symptoms of rice blast disease, I should search for information related to crop diseases, specifically rice blast disease. Since I have access to the AgroPulse blog database through the CropDocs function, I can use this to find detailed information about crop diseases.

Action: CropDocs
Action Input: rice blast disease symptoms
Observation: An error occurred: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01k2vdfmwef639kq28wedexz4p` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 100204, Requested 834. Please try again in 14m57.604999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Thought:

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01k2vdfmwef639kq28wedexz4p` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 100204, Requested 538. Please try again in 10m41.788s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:
print(agent.run("And what is the best fertilizer for rice?"))



> Entering new AgentExecutor chain...
To answer this question, I need to consider the specific nutritional requirements of rice crops and the various types of fertilizers available.

Action: CropDocs
Action Input: "best fertilizer for rice"
Observation: I don't know the specific best fertilizer for rice as the provided context only mentions "Essential Fertilizers for Maximum Yield" in general, without providing details on the type of fertilizer or specific recommendations for rice.
Thought:Since the initial observation didn't provide a specific answer, I'll try a broader search to gather more information about fertilizers used for rice crops.

Action: wikipedia
Action Input: "rice fertilizer"
Observation: Page: Rice hull
Summary: Rice hulls or husks are the hard protecting coverings of grains of rice. In addition to protecting rice during the growing season, rice hulls can be put to use as building material, fertilizer, insulation material, or fuel. R
Thought:It seems like the observ

In [ ]:
print(agent.run("Remind me what we discussed about rice blast earlier."))



> Entering new AgentExecutor chain...
Thought: I don't have any record of a previous conversation about rice blast, so I'll need to start from scratch. To provide a reminder about what we discussed earlier, I would typically need to access a database or knowledge base that stores our previous conversations. However, since I don't have that capability, I can try to provide general information about rice blast.

Action: CropDocs
Action Input: rice blast
Observation: The provided context does not specifically mention "rice blast." However, since one of the blog posts is about "How to Master Paddy Cultivation in India: Climate-Smart Tips & Insurance Guide," it can be inferred that the topic might be related to paddy cultivation, which includes rice.

Rice blast is a fungal disease that affects rice plants, caused by the fungus Magnaporthe oryzae. It is a significant problem in rice cultivation worldwide, leading to reduced yields and lower quality rice. 

If you're looking for informatio

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `Thought: I was able to find some general information about rice blast, but I realize that I didn't actually have any record of a previous conversation about the topic. Since I'm a large language model, I don't have the ability to retain information about previous conversations or maintain a memory of individual users. Each time a user interacts with me, it's a new conversation.

Action: None`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 